In [22]:
import torch
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained('gpt2')

In [ ]:
import wandb

In [ ]:
run = wandb.init(project="Head Word Final 2")

In [ ]:
api = wandb.Api()

In [ ]:
runs = {}
for run in api.runs(
    path="ananthag/Head Word Final 2"
):
    runs[run.config['model_name']] = run.id

In [36]:
tok.convert_tokens_to_ids('bite')

37018

In [5]:
tok.decode(4532)

CodeGenTokenizerFast(name_or_path='microsoft/phi-2', vocab_size=50257, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50257: AddedToken("                               ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50258: AddedToken("                              ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50259: AddedToken("                             ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50260: AddedToken("                            ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50261: AddedToken("            

In [13]:
# tok.encode(' adjust', add_prefix_space=False)
type(tok)

transformers.models.codegen.tokenization_codegen_fast.CodeGenTokenizerFast

In [23]:
not tok.pad_token_id

True

In [14]:
inputs = ["Hey what's your name", "Welcome home", "text" * 20]
tok.pad_token_id = tok.eos_token_id
inputs_dict = tok.batch_encode_plus(inputs, padding=True, return_tensors='pt')
from transformers import AutoModelForCausalLM

m = AutoModelForCausalLM.from_pretrained('gpt2')
out = m(**inputs_dict)

In [20]:
tok.pad_token_id

50256

In [83]:
inputs = inputs_dict["input_ids"]

In [57]:
attn_mask = inputs_dict["attention_mask"]

last_non_masked_idx = torch.sum(attn_mask, dim=1) - 1

In [49]:
position_ids = torch.tensor(
            [list(range(inputs.shape[1])) for i in range(inputs.shape[0])]
        )

In [61]:
for i, position_ids_slice in enumerate(position_ids):
    position_ids_slice[last_non_masked_idx[i] :] = position_ids_slice[
        last_non_masked_idx[i]
    ]

In [50]:
torch.arange(inputs.shape[1]).repeat(inputs.shape[0], 1)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19]])

In [64]:
inputs[attn_mask] = 2

In [66]:
attn_mask

tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [80]:

inputs.scatter_(0, 1 - attn_mask, last_non_masked_idx.unsqueeze(1))
inputs


RuntimeError: Expected index [3, 20] to be smaller than self [3, 20] apart from dimension 0 and to be smaller size than src [3, 1]

In [98]:
inputs = inputs * attn_mask + (~attn_mask.bool() * last_non_masked_idx.unsqueeze(1).expand(-1, inputs.shape[1]))

tensor([[False, False, False, False, False,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False]])

In [99]:
inputs

tensor([[10814,   644,   338,   534,  1438,     4,     4,     4,     4,     4,
             4,     4,     4,     4,     4,     4,     4,     4,     4,     4],
        [14618,  1363,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [ 5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239,
          5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239,  5239]])

In [2]:
import dataset
from dataset import DepParseDataPickle

d = dataset.HeadWordDatasetWithRelns('test', 'gpt2', 12, 768)

In [9]:
len(d.start_indices)

2416

In [10]:
d.start_indices[2415]

58223

In [12]:
d.hidden_state_cache.shape

(58236, 12, 768)

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
X = [[0, 0, 0], [1, 1, 1]]
Y = [[1, 0, 0], [1, 1, 0]]
cosine_similarity(Y, Y)

array([[1.        , 0.70710678],
       [0.70710678, 1.        ]])

In [19]:
import numpy as np

ran = np.random.rand(3, 3)
ran[np.eye(3, 3, dtype=bool)] = 0

In [23]:
ran.argmax(axis=1)

array([3, 3, 2])

In [16]:
import csv
import collections
counter = collections.Counter()
models = set()
with open('surprisals_errata.csv') as f:
    reader = csv.DictReader(f)
    for row in reader:
        counter[(row['file_name'], row['item_number'])] += 1
        models.add(row['model'])

len(models)        

15

In [15]:
counter.most_common(100)

[(('reflexive_src_fem.json', '6'), 15),
 (('reflexive_src_fem.json', '7'), 15),
 (('reflexive_src_fem.json', '13'), 15),
 (('reflexive_src_fem.json', '16'), 15),
 (('reflexive_prep_fem.json', '1'), 15),
 (('reflexive_prep_fem.json', '2'), 15),
 (('reflexive_src_fem.json', '1'), 14),
 (('reflexive_src_fem.json', '5'), 14),
 (('reflexive_src_fem.json', '15'), 14),
 (('reflexive_src_fem.json', '19'), 14),
 (('reflexive_prep_fem.json', '17'), 14),
 (('fgd-embed3.json', '4'), 14),
 (('fgd_subject.json', '4'), 14),
 (('fgd_subject.json', '5'), 14),
 (('fgd_subject.json', '9'), 14),
 (('fgd_subject.json', '10'), 14),
 (('mvrr.json', '5'), 14),
 (('mvrr_mod.json', '5'), 14),
 (('number_orc.json', '5'), 14),
 (('number_orc.json', '17'), 14),
 (('number_prep.json', '5'), 14),
 (('number_prep.json', '17'), 14),
 (('number_src.json', '5'), 14),
 (('number_src.json', '17'), 14),
 (('reflexive_orc_fem.json', '1'), 14),
 (('reflexive_orc_fem.json', '5'), 14),
 (('reflexive_orc_fem.json', '6'), 14),
 

In [1]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForCausalLM,
    GPTNeoXTokenizerFast,
    LlamaTokenizer,
    GPT2Tokenizer,
    GemmaTokenizer,
)


In [3]:
batch = ["cat", "dog", "cow", "bird"]
model_name = 'google/gemma-2b'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast='pythia' in 'google')
# tokenizer.pad_token_id = tokenizer.eos_token_id
# batch = tokenizer(("one word", "two two two words", "three three three three three words"), padding=True, return_tensors='pt')

# model = AutoModelForCausalLM.from_pretrained('gpt2-large')
# output = model(**batch)
tokenizer

GemmaTokenizerFast(name_or_path='google/gemma-2b', vocab_size=256000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<eos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<bos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [17]:
batch

{'input_ids': tensor([[  505,  1573, 50256, 50256, 50256, 50256],
        [11545,   734,   734,  2456, 50256, 50256],
        [15542,  1115,  1115,  1115,  1115,  2456]]), 'attention_mask': tensor([[1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1]])}

In [18]:
batch["input_ids"]

tensor([[  505,  1573, 50256, 50256, 50256, 50256],
        [11545,   734,   734,  2456, 50256, 50256],
        [15542,  1115,  1115,  1115,  1115,  2456]])

In [14]:
batch.attention_mask.unsqueeze(-1)

tensor([[[1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [0]],

        [[1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1]]])

In [24]:
import torch.nn.functional as F
targets = torch.cat((batch["input_ids"][:, 1:], torch.tensor([tokenizer.eos_token_id]).expand(batch["input_ids"].shape[0], 1)), dim=1)
out = torch.gather(F.log_softmax(output.logits, dim=-1), -1, targets.unsqueeze(-1))
(out * batch.attention_mask.unsqueeze(-1)).view(batch.input_ids.shape[0], -1).sum(-1)

tensor([-17.2175, -27.2265, -33.0799], grad_fn=<SumBackward1>)

In [26]:
import os
os.listdir("../blimp/data")

['animate_subject_passive.jsonl',
 'principle_A_domain_3.jsonl',
 'npi_present_1.jsonl',
 'wh_vs_that_no_gap_long_distance.jsonl',
 'existential_there_quantifiers_1.jsonl',
 'determiner_noun_agreement_irregular_1.jsonl',
 'ellipsis_n_bar_1.jsonl',
 'causative.jsonl',
 'wh_questions_object_gap.jsonl',
 'transitive.jsonl',
 'superlative_quantifiers_1.jsonl',
 'tough_vs_raising_2.jsonl',
 'principle_A_case_1.jsonl',
 'expletive_it_object_raising.jsonl',
 'irregular_plural_subject_verb_agreement_2.jsonl',
 'principle_A_domain_2.jsonl',
 'sentential_negation_npi_scope.jsonl',
 'ellipsis_n_bar_2.jsonl',
 'determiner_noun_agreement_irregular_2.jsonl',
 'left_branch_island_echo_question.jsonl',
 'existential_there_quantifiers_2.jsonl',
 'wh_island.jsonl',
 'superlative_quantifiers_2.jsonl',
 'principle_A_domain_1.jsonl',
 'irregular_plural_subject_verb_agreement_1.jsonl',
 'distractor_agreement_relative_clause.jsonl',
 'existential_there_subject_raising.jsonl',
 'tough_vs_raising_1.jsonl',
 'p

In [34]:
targets = torch.cat(
    (
        batch.input_ids[:, 1:],
        torch.tensor([[tokenizer.eos_token_id]]).expand(
            batch.input_ids.shape[0], 1
        ),
    ),
    dim=-1,
)
logprobs = torch.gather(
    torch.nn.functional.log_softmax(output.logits, dim=-1),
    dim=-1,
    index=targets.unsqueeze(1),
).reshape(-1)
logprobs

tensor([-6.4905, -6.7212, -6.6658, -6.3376], grad_fn=<ViewBackward0>)

In [37]:
torch.sum(logprobs > logprobs)

tensor(0)

In [21]:
torch.tensor([[tokenizer.eos_token_id]]).expand(4, 1)

tensor([[50256],
        [50256],
        [50256],
        [50256]])

In [3]:
AutoModelForCausalLM.from_pretrained('google/gemma-2b').config

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GemmaConfig {
  "_name_or_path": "google/gemma-2b",
  "architectures": [
    "GemmaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "head_dim": 256,
  "hidden_act": "gelu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 16384,
  "max_position_embeddings": 8192,
  "model_type": "gemma",
  "num_attention_heads": 8,
  "num_hidden_layers": 18,
  "num_key_value_heads": 1,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 256000
}